In [65]:
import pandas as pd, numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import keras
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import confusion_matrix

In [66]:
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 
           'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']

df = pd.read_csv('pima-indians-diabetes.csv', names = columns)
df.shape


(768, 9)

In [67]:
#let's check if there are any na values in any column

df.isna().any() 

Pregnancies                 False
Glucose                     False
BloodPressure               False
SkinThickness               False
Insulin                     False
BMI                         False
DiabetesPedigreeFunction    False
Age                         False
Outcome                     False
dtype: bool

In [68]:
df[df == 0].head() # there are zeros in data let's replace them by the column mean

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0
2,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
for col in df.columns[:-1]:
    df[col].replace(to_replace=0, value=np.mean(df[col]), inplace=True)
    
df.head(10) # replaces zero values in features by their column means

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6.000000,148.0,72.000000,35.000000,79.799479,33.600000,0.627,50,1
1,1.000000,85.0,66.000000,29.000000,79.799479,26.600000,0.351,31,0
2,8.000000,183.0,64.000000,20.536458,79.799479,23.300000,0.672,32,1
3,1.000000,89.0,66.000000,23.000000,94.000000,28.100000,0.167,21,0
4,3.845052,137.0,40.000000,35.000000,168.000000,43.100000,2.288,33,1
5,5.000000,116.0,74.000000,20.536458,79.799479,25.600000,0.201,30,0
6,3.000000,78.0,50.000000,32.000000,88.000000,31.000000,0.248,26,1
7,10.000000,115.0,69.105469,20.536458,79.799479,35.300000,0.134,29,0
8,2.000000,197.0,70.000000,45.000000,543.000000,30.500000,0.158,53,1
9,8.000000,125.0,96.000000,20.536458,79.799479,31.992578,0.232,54,1


In [70]:
# let's separate the features from labels

X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

X.shape, y.shape

((768, 8), (768,))

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, random_state = 0)

X_train.shape

(537, 8)

In [72]:
# no need for label encoding for features and labels
# feature scaling

mm_X = MinMaxScaler()
mm_X.fit_transform(X_train)
mm_X.transform(X_test)

array([[0.        , 1.00704225, 0.5       , ..., 0.50511247, 0.56078767,
        0.01666667],
       [0.0625    , 0.35915493, 0.47826087, ..., 0.31492843, 0.1369863 ,
        0.03333333],
       [0.1875    , 0.14084507, 0.34782609, ..., 0.32310838, 0.13142123,
        0.06666667],
       ...,
       [0.        , 0.27464789, 0.32608696, ..., 0.11656442, 0.07534247,
        0.01666667],
       [0.        , 0.35211268, 0.5       , ..., 0.39468303, 0.04837329,
        0.08333333],
       [0.5625    , 0.46478873, 0.52173913, ..., 0.19222904, 0.18321918,
        0.4       ]])

Use 1 output node. Output 0 (<0.5) is considered class A and 1 (>=0.5) is considered class B (in case of sigmoid)

Use 2 output nodes. The input belongs to the class of the node with the highest value/probability (argmax).

In the second case you are probably writing about softmax activation function. If that's true, than the sigmoid is just a special case of softmax function. That's easy to show.

$$y=1/(1+e^x)=1/(1+1/e^x)=1/((e^x+1)/e^x)=e^x/(1+e^x)=e^x/(e^0+e^x)$$

As you can see sigmoid is the same as softmax. You can think that you have two outputs, but one of them has all weights equal to zero and therefore its output will be always equal to zero.

So the better choice for the binary classification is to use one output unit with sigmoid instead of softmax with two output units, because it will update faster.

In [77]:
EPOCHS = 500
batch_size = 50

def model_baseline():
    model = Sequential()
    model.add(Dense(input_dim = 8, units = 10, activation = 'relu',
                    kernel_initializer = 'uniform'))
#     model.add(Dropout(0.1))
    model.add(Dense(units = 10, activation = 'relu',
                    kernel_initializer = 'uniform'))
#     model.add(Dropout(0.1))
    model.add(Dense(units = 1, activation = 'sigmoid',
                    kernel_initializer = 'uniform'))
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model    
            
stratified_kfold = StratifiedKFold(n_splits=5)
classifier = KerasClassifier(build_fn = model_baseline, epochs = EPOCHS, batch_size = batch_size)
classifier.fit(X_train, y_train)
              
results = cross_val_score(classifier, X_train, y_train, cv = stratified_kfold, n_jobs = -1, scoring = 'accuracy')
print(results)
print()
print('The mean accuracy is: {}%'.format(np.mean(results) * 100))
print('The variance is: {}'.format(np.var(results)))
              

Epoch 1/500
537/537 [==============================] - 0s 257us/step - loss: 0.6921 - accuracy: 0.6313
Epoch 2/500
537/537 [==============================] - 0s 39us/step - loss: 0.6890 - accuracy: 0.6387
Epoch 3/500
537/537 [==============================] - 0s 41us/step - loss: 0.6826 - accuracy: 0.6387
Epoch 4/500
537/537 [==============================] - 0s 54us/step - loss: 0.6772 - accuracy: 0.6387
Epoch 5/500
537/537 [==============================] - 0s 52us/step - loss: 0.6714 - accuracy: 0.6387
Epoch 6/500
537/537 [==============================] - 0s 54us/step - loss: 0.6668 - accuracy: 0.6387
Epoch 7/500
537/537 [==============================] - 0s 47us/step - loss: 0.6633 - accuracy: 0.6387
Epoch 8/500
537/537 [==============================] - 0s 47us/step - loss: 0.6581 - accuracy: 0.6443
Epoch 9/500
537/537 [==============================] - 0s 47us/step - loss: 0.6564 - accuracy: 0.6406
Epoch 10/500
537/537 [==============================] - 0s 34us/step - loss: 0.65

537/537 [==============================] - 0s 32us/step - loss: 0.5402 - accuracy: 0.7412
Epoch 81/500
537/537 [==============================] - 0s 32us/step - loss: 0.5385 - accuracy: 0.7337
Epoch 82/500
537/537 [==============================] - 0s 35us/step - loss: 0.5381 - accuracy: 0.7393
Epoch 83/500
537/537 [==============================] - 0s 32us/step - loss: 0.5427 - accuracy: 0.7318
Epoch 84/500
537/537 [==============================] - 0s 30us/step - loss: 0.5396 - accuracy: 0.7412
Epoch 85/500
537/537 [==============================] - 0s 30us/step - loss: 0.5412 - accuracy: 0.7486
Epoch 86/500
537/537 [==============================] - 0s 32us/step - loss: 0.5512 - accuracy: 0.7114
Epoch 87/500
537/537 [==============================] - 0s 35us/step - loss: 0.5592 - accuracy: 0.7095
Epoch 88/500
537/537 [==============================] - 0s 34us/step - loss: 0.5510 - accuracy: 0.7318
Epoch 89/500
537/537 [==============================] - 0s 32us/step - loss: 0.5456 - 

537/537 [==============================] - 0s 39us/step - loss: 0.5173 - accuracy: 0.7393
Epoch 159/500
537/537 [==============================] - 0s 39us/step - loss: 0.5157 - accuracy: 0.7542
Epoch 160/500
537/537 [==============================] - 0s 35us/step - loss: 0.5163 - accuracy: 0.7467
Epoch 161/500
537/537 [==============================] - 0s 37us/step - loss: 0.5161 - accuracy: 0.7505
Epoch 162/500
537/537 [==============================] - 0s 35us/step - loss: 0.5210 - accuracy: 0.7393
Epoch 163/500
537/537 [==============================] - 0s 37us/step - loss: 0.5204 - accuracy: 0.7467
Epoch 164/500
537/537 [==============================] - 0s 39us/step - loss: 0.5135 - accuracy: 0.7374
Epoch 165/500
537/537 [==============================] - 0s 34us/step - loss: 0.5205 - accuracy: 0.7505
Epoch 166/500
537/537 [==============================] - 0s 33us/step - loss: 0.5173 - accuracy: 0.7393
Epoch 167/500
537/537 [==============================] - 0s 30us/step - loss: 

537/537 [==============================] - 0s 35us/step - loss: 0.4920 - accuracy: 0.7672
Epoch 237/500
537/537 [==============================] - 0s 37us/step - loss: 0.4906 - accuracy: 0.7709
Epoch 238/500
537/537 [==============================] - 0s 39us/step - loss: 0.4938 - accuracy: 0.7523
Epoch 239/500
537/537 [==============================] - 0s 32us/step - loss: 0.4966 - accuracy: 0.7654
Epoch 240/500
537/537 [==============================] - 0s 32us/step - loss: 0.4924 - accuracy: 0.7616
Epoch 241/500
537/537 [==============================] - 0s 32us/step - loss: 0.4943 - accuracy: 0.7672
Epoch 242/500
537/537 [==============================] - 0s 35us/step - loss: 0.5044 - accuracy: 0.7486
Epoch 243/500
537/537 [==============================] - 0s 30us/step - loss: 0.4975 - accuracy: 0.7561
Epoch 244/500
537/537 [==============================] - 0s 32us/step - loss: 0.4931 - accuracy: 0.7728
Epoch 245/500
537/537 [==============================] - 0s 32us/step - loss: 

537/537 [==============================] - 0s 41us/step - loss: 0.4750 - accuracy: 0.7765
Epoch 315/500
537/537 [==============================] - 0s 39us/step - loss: 0.4727 - accuracy: 0.7672
Epoch 316/500
537/537 [==============================] - 0s 41us/step - loss: 0.4801 - accuracy: 0.7654
Epoch 317/500
537/537 [==============================] - 0s 50us/step - loss: 0.4728 - accuracy: 0.7765
Epoch 318/500
537/537 [==============================] - 0s 41us/step - loss: 0.4711 - accuracy: 0.7672
Epoch 319/500
537/537 [==============================] - 0s 33us/step - loss: 0.4739 - accuracy: 0.7654
Epoch 320/500
537/537 [==============================] - 0s 39us/step - loss: 0.4805 - accuracy: 0.7654
Epoch 321/500
537/537 [==============================] - 0s 35us/step - loss: 0.4872 - accuracy: 0.7654
Epoch 322/500
537/537 [==============================] - 0s 47us/step - loss: 0.4868 - accuracy: 0.7635
Epoch 323/500
537/537 [==============================] - 0s 35us/step - loss: 

537/537 [==============================] - 0s 35us/step - loss: 0.4558 - accuracy: 0.7728
Epoch 393/500
537/537 [==============================] - 0s 48us/step - loss: 0.4548 - accuracy: 0.7691
Epoch 394/500
537/537 [==============================] - 0s 35us/step - loss: 0.4603 - accuracy: 0.7672
Epoch 395/500
537/537 [==============================] - 0s 37us/step - loss: 0.4547 - accuracy: 0.7709
Epoch 396/500
537/537 [==============================] - 0s 43us/step - loss: 0.4545 - accuracy: 0.7691
Epoch 397/500
537/537 [==============================] - 0s 63us/step - loss: 0.4548 - accuracy: 0.7728
Epoch 398/500
537/537 [==============================] - 0s 47us/step - loss: 0.4555 - accuracy: 0.7728
Epoch 399/500
537/537 [==============================] - 0s 45us/step - loss: 0.4527 - accuracy: 0.7803
Epoch 400/500
537/537 [==============================] - 0s 39us/step - loss: 0.4541 - accuracy: 0.7747
Epoch 401/500
537/537 [==============================] - 0s 39us/step - loss: 

537/537 [==============================] - 0s 39us/step - loss: 0.4545 - accuracy: 0.7803
Epoch 471/500
537/537 [==============================] - 0s 37us/step - loss: 0.4483 - accuracy: 0.7858
Epoch 472/500
537/537 [==============================] - 0s 39us/step - loss: 0.4420 - accuracy: 0.7821
Epoch 473/500
537/537 [==============================] - 0s 35us/step - loss: 0.4412 - accuracy: 0.7821
Epoch 474/500
537/537 [==============================] - 0s 37us/step - loss: 0.4513 - accuracy: 0.7803
Epoch 475/500
537/537 [==============================] - 0s 37us/step - loss: 0.4440 - accuracy: 0.7747
Epoch 476/500
537/537 [==============================] - 0s 32us/step - loss: 0.4458 - accuracy: 0.7821
Epoch 477/500
537/537 [==============================] - 0s 35us/step - loss: 0.4480 - accuracy: 0.7784
Epoch 478/500
537/537 [==============================] - 0s 33us/step - loss: 0.4478 - accuracy: 0.7803
Epoch 479/500
537/537 [==============================] - 0s 43us/step - loss: 

Tried my best to brute force every hyperparameter manually each time instead running a grid search because I don't think my laptop will get that done in minutes :D

P.S: this is not the best accuracy I got but I just stuck to this one.

In [78]:
# low variance so I do not need to use to Dropout

predictions = classifier.predict(X_test)
df_pred = pd.Series(predictions.flatten())
df_actual = pd.Series(y_test)

print(df_pred.value_counts())
df_actual.value_counts()

0    171
1     60
dtype: int64


0    157
1     74
dtype: int64

In [79]:
cm = confusion_matrix(y_test, predictions)

cm_df = pd.DataFrame(data=cm, columns=[['predicted','predicted'], ['Not Diabetic','Diabetic']],
                    index=[['actual','actual'], ['Not Diabetic','Diabetic']])

cm_df

predicted         
                    Not Diabetic Diabetic
actual Not Diabetic          138       19
       Diabetic               33       41

In [80]:
test_accuracy = ((138+41)/y_test.shape[0]) * 100
print('The accuracy of predicting the test set is {}%'.format(test_accuracy))

The accuracy of predicting the test set is 77.48917748917748%
